In [ ]:
import json 
import requests

# Submit Job

In [7]:
job_para = json.load(open('data/pat_job_test.json'))
job_para

{'job_guid': '80a1ca07-41f9-4f34-b7bb-d01b477bc78d',
 'job_name': 'PAT_Test_AIG',
 'type_of_rating': 'PSOLD',
 'coverage': 'Building + Contents + Time Element',
 'peril_subline': 'All_Perils',
 'subject_premium': 100000000.0,
 'loss_alae_ratio': 1,
 'average_accident_date': '1/1/2022',
 'trend_factor': 1.035,
 'additional_coverage': 2,
 'deductible_treatment': 'Retains Limit',
 'server': 'DFWCAT-RMS5SQL1',
 'edm_database': 'RMS_EPL_AIG_PAT_Testing_202107_EDM181',
 'rdm_database': 'RMS_EPL_AIG_PAT_Testing_202107_RDM181',
 'portinfoid': 6,
 'perilid': 5,
 'analysisid': 5,
 'user_name': 'cxiao',
 'user_ema': 'chengyou.xiao@guycarp.com'}

In [6]:
files = {'para': ('pat_para', open('data/pat_job_test.json'))}
ret = requests.post("http://localhost:5000/api/Jobs", files = files)
ret.status_code, ret.content

(200, b'Analysis submitted!')

# Results Summary